In [62]:
import pandas as pd

order_report_df = pd.read_excel('Company X - Order Report.xlsx')
sku_master_df = pd.read_excel('Company X - SKU Master.xlsx')
pincode_zones_df = pd.read_excel('Company X - Pincode Zones.xlsx')
invoice_df = pd.read_excel('Courier Company - Invoice.xlsx')
rates_df = pd.read_excel('Courier Company - Rates.xlsx')

In [63]:
order_report_df.head()

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0


In [64]:
sku_master_df.head()

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


In [65]:
pincode_zones_df.head()


,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d


In [66]:
invoice_df.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4


In [67]:
rates_df.head()

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,A,0.50,29.5,23.6,13.6,23.6
1,B,1.00,33.0,28.3,20.5,28.3
2,C,1.25,40.1,38.9,31.9,38.9
3,D,1.50,45.4,44.8,41.3,44.8
4,E,2.00,56.6,55.5,50.7,55.5


### Cleaning Data-

In [68]:
order_report_df = pd.read_excel('Company X - Order Report.xlsx')
sku_master_df = pd.read_excel('Company X - SKU Master.xlsx')
pincode_zones_df = pd.read_excel('Company X - Pincode Zones.xlsx')
invoice_df = pd.read_excel('Courier Company - Invoice.xlsx')
rates_df = pd.read_excel('Courier Company - Rates.xlsx')
order_report_df.dropna(subset=['Order Qty'], inplace=True)
order_report_df['ExternOrderNo'] = order_report_df['ExternOrderNo'].astype(str)
sku_master_df.dropna(subset=['Weight (g)'], inplace=True)
sku_master_df['Weight (kg)'] = sku_master_df['Weight (g)'] / 1000
sku_master_df.drop(columns=['Weight (g)'], inplace=True)
pincode_zones_df.dropna(subset=['Zone'], inplace=True)
invoice_df.dropna(subset=['Charged Weight', 'Billing Amount (Rs.)'], inplace=True)
invoice_df['Order ID'] = invoice_df['Order ID'].astype(str)

order_sku_merged_df = order_report_df.merge(sku_master_df, on='SKU')

order_sku_merged_df['Total Weight per SKU (kg)'] = order_sku_merged_df['Order Qty'] * order_sku_merged_df['Weight (kg)']


order_total_weight_df = order_sku_merged_df.groupby('ExternOrderNo', as_index=False).agg({'Total Weight per SKU (kg)': 'sum'})
order_total_weight_df.rename(columns={'Total Weight per SKU (kg)': 'Total Order Weight (kg)'}, inplace=True)


order_pincode_df = invoice_df[['Order ID', 'Warehouse Pincode', 'Customer Pincode']].drop_duplicates()
order_pincode_df.rename(columns={'Order ID': 'ExternOrderNo'}, inplace=True)


order_total_weight_pincodes_df = order_total_weight_df.merge(order_pincode_df, on='ExternOrderNo', how='left')


final_df = order_total_weight_pincodes_df.merge(pincode_zones_df, on=['Warehouse Pincode', 'Customer Pincode'], how='left')


final_df = final_df[['ExternOrderNo', 'Total Order Weight (kg)', 'Zone']]


final_output_path = 'integrated_data.xlsx'
final_df.to_excel(final_output_path, index=False)

final_output_path

'integrated_data.xlsx'

In [69]:
final_data_df = pd.read_excel('integrated_data.xlsx')

In [70]:
final_data_df.head()

,ExternOrderNo,Total Order Weight (kg),Zone
0,2001806210,0.220,b
1,2001806226,0.480,d
2,2001806229,0.500,d
3,2001806232,1.302,d
4,2001806233,0.245,b


In [71]:
def calculate_expected_charge(weight, zone, includes_rto):
    # Filter the rates for the given zone
    zone_rates = rates_df[rates_df['Zone'] == zone.upper()].iloc[0]
    
 
    slabs_count = weight / zone_rates['Weight Slabs']
    full_slabs = int(slabs_count)
    partial_slab = slabs_count - full_slabs
    
   
    if includes_rto:
        fixed_charge = zone_rates['RTO Fixed Charge']
        additional_charge = zone_rates['RTO Additional Weight Slab Charge']
    else:
        fixed_charge = zone_rates['Forward Fixed Charge']
        additional_charge = zone_rates['Forward Additional Weight Slab Charge']
    
    
    total_charge = fixed_charge
    if full_slabs > 1: 
        total_charge += (full_slabs - 1) * additional_charge
    
   
    if partial_slab > 0:
        total_charge += additional_charge
    
    return total_charge


calculate_expected_charge(weight=2.25, zone='C', includes_rto=False)

79.0

In [73]:

final_data_df = pd.read_excel('final_integrated_data.xlsx')


invoice_df = pd.read_excel('Courier Company - Invoice.xlsx')


merged_data = final_data_df.merge(invoice_df, left_on='ExternOrderNo', right_on='Order ID', how='left')


merged_data['Includes RTO'] = merged_data['Type of Shipment'].str.contains('rto', case=False, na=False)
merged_data['Expected Charge (Rs.)'] = merged_data.apply(
    lambda row: calculate_expected_charge(
        weight=row['Total Order Weight (kg)'],
        zone=row['Zone_x'], 
        includes_rto=row['Includes RTO']
    ),
    axis=1
)


merged_data['Difference'] = merged_data['Expected Charge (Rs.)'] - merged_data['Billing Amount (Rs.)']

merged_data['Charge Classification'] = 'Correctly Charged'
merged_data.loc[merged_data['Difference'] > 0, 'Charge Classification'] = 'Undercharged'
merged_data.loc[merged_data['Difference'] < 0, 'Charge Classification'] = 'Overcharged'

# Create the summary table
summary_table = merged_data['Charge Classification'].value_counts().rename_axis('Status').reset_index(name='Count')
summary_table['Total Amount (Rs.)'] = summary_table.apply(
    lambda row: merged_data.loc[merged_data['Charge Classification'] == row['Status'], 'Difference'].sum(),
    axis=1
)


total_invoice_amount_correct = merged_data.loc[merged_data['Charge Classification'] == 'Correctly Charged', 'Billing Amount (Rs.)'].sum()


summary_table.loc[summary_table['Status'] == 'Correctly Charged', 'Total Amount (Rs.)'] = total_invoice_amount_correct



order_level_calculation.rename(columns={
    'Zone_x': 'Delivery Zone as per X',
    'Charged Weight': 'Weight slab charged by Courier Company (KG)',
    'Total Order Weight (kg)': 'Total weight as per X (KG)',
    'Type of Shipment': 'Type of Shipment',
    'Expected Charge (Rs.)': 'Expected Charge as per X (Rs.)',
    'Billing Amount (Rs.)': 'Charges Billed by Courier Company (Rs.)',
    'Difference': 'Difference Between Expected Charges and Billed Charges (Rs.)'
}, inplace=True)


with pd.ExcelWriter('TASK-OUTPUT-FILE.xlsx') as writer:  
    order_level_calculation.to_excel(writer, sheet_name='Order Level Calculation', index=False)
    summary_table.to_excel(writer, sheet_name='Summary Table', index=False)


print('TASK-OUTPUT-FILE.xlsx')



TASK-OUTPUT-FILE.xlsx
